# RAG Antibiotics Chat

In [1]:
import os
import uuid
import time

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_ollama import ChatOllama
import chromadb
from chromadb.config import Settings

In [2]:
# Embedding Model

model = SentenceTransformer("all-MiniLM-L6-v2")

In [3]:
# Load Documents
def load_split_documents(path):
    loader = PyMuPDFLoader(path,
        extract_tables="markdown",
    )
    docs = loader.load()

    recursive_splitter = RecursiveCharacterTextSplitter(
        separators="\n",
        chunk_size=512,
        chunk_overlap=20
    )

    # chunked documents
    chunked_documents = recursive_splitter.split_documents(docs)

    return chunked_documents


In [4]:

# Vector Database
def add_vector_chroma(col_name, db_name, chunked_documents):

    sentences = [doc.page_content for doc in chunked_documents]

    metadatas = [{"titulo": doc.metadata.get("title", "")} for doc in chunked_documents]

    embeddings = model.encode(sentences).tolist()

    client = chromadb.PersistentClient(path=db_name)

    collection = client.get_or_create_collection(name=col_name)
    
    ids = [str(uuid.uuid4()) for _ in sentences]
    collection.add(
        ids=ids,
        documents=sentences,
        embeddings=embeddings,
        metadatas = metadatas
    )



In [5]:
# Percorre todos os arquivos do diretório

path = "pdf/"

for arquivo in os.listdir(path):
    if arquivo.lower().endswith('.pdf'):  
        path_pdf = os.path.join(path, arquivo)
        chunked_documents = load_split_documents(path_pdf)
        add_vector_chroma("database", "db", chunked_documents)

## Retrieve ChromaDB

In [6]:
#Initialize chroma db

def initialize_chromadb(db_name):
    
    client = chromadb.PersistentClient(path=db_name) 

    return client

In [7]:
# Instantiate Vector Database

client = initialize_chromadb("db")

# Listando as collections existentes
collections = client.list_collections()

# Recuperando a primeira coleção
col = client.get_or_create_collection(collections[0].name)

# Verificando a quantidade de itens na coleção
col.count()

945

In [8]:
# Retriver do banco vetorial do Chroma

def chroma_retriever(query, top_k=8):
    
    query_vec = model.encode(query)
   
    result = col.query(
        query_embeddings=[query_vec],
        n_results=top_k
    )
    return result["documents"][0]

## LLM and Answers

In [9]:
from langchain_core.messages import ChatMessage, SystemMessage, HumanMessage

In [24]:
system_prompt = """ You are a doctor's answering task 
    assistant. Use the following parts of 
    the retrieved context to answer the 
    question. Response according to 
    context. Context: {context}
    Translate the answer 
    to portuguese. 
    """

In [25]:
MODEL_LLAMA = "llama3.2:3b"
MODEL_GEMMA = "gemma3:1b"
MODEL_DEEPSEEK = "deepseek-r1:1.5b"

# declare the model
llm = ChatOllama(
    model= MODEL_LLAMA,
    temperature=0.1
)

In [26]:
def generate_answer(llm, question):
    
    context = chroma_retriever(question)
    
    system_prompt_fmt = system_prompt.format(context=context)

    response = llm.invoke([SystemMessage(content=system_prompt_fmt),
                          HumanMessage(content=question)])
    
    return response.content

## Testes

In [27]:
inicio = time.time()

print(generate_answer(llm, "What Is the Role of Combination Antibiotic Therapy for the Treatment of Infections Caused by DTR P. aeruginosa?"))

fim = time.time()
print(f"Tempo de execução: {fim - inicio:.4f} segundos")

A combinação de antibióticos não é sugerida para infecções causadas por P. aeruginosa com resistência a DTR (Derechos de Tratamiento Resistente) se a suscetibilidade a ceftolozane-tazobactam, ceftazidime-avibactam, imipenem-cilastatin-relebactam ou cefiderocol forneceu resultados positivos.

Em vez disso, o tratamento sugere que, quando os isolados de P. aeruginosa são suscetíveis a agentes não-carbapenêmicos tradicionais (como piperacilina-tazobactam, ceftazidime e cefepime), a terapia com antibióticos aminoglucosídeos e polimixinas pode ser considerada. Além disso, se os isolados de P. aeruginosa são suscetíveis a um agente β-lactâmico, a combinação de tobramicina com esse agente não oferece benefícios adicionais sobre o tratamento com monoterapia com o agente β-lactâmico.

Se nenhum agente β-lactâmico demonstrar atividade contra P. aeruginosa com resistência a DTR, a tobramicina pode ser considerada como uma opção alternativa para o tratamento de infecções urinárias ou cistite uriná

In [28]:
inicio = time.time()

print(generate_answer(llm, "What is the indication and dosage of Levofloxacin and Minocycline according to the document?"))

fim = time.time()
print(f"Tempo de execução: {fim - inicio:.4f} segundos")

Infelizmente, o documento não menciona a Levofloxacina como opção de tratamento. No entanto, posso fornecer informações sobre a Indicação e Dose de Minociclina e Tigeciclina, que são as duas opções apresentadas no documento.

**Minociclina:**

* Indicação: Infeções bacterianas graves, incluindo sepsis, meningite, osteomielite, etc.
* Dose:
 + Sulbactam 1 g IV a cada 4 horas por 3 horas
 + Durlobactam 1 g IV a cada 4 horas por 3 horas (em combinação com sulbactam)
 + Minociclina 200 mg IV como dose única, seguida de 100 mg IV a cada 12 horas

**Tigeciclina:**

* Indicação: Infeções bacterianas graves que são resistentes ou não respondem a outros antibióticos.
* Dose:
 + 200 mg IV como dose única, seguida de 100 mg IV a cada 12 horas
Tempo de execução: 57.9171 segundos


In [29]:
inicio = time.time()

print(generate_answer(llm, "What Is the General Approach for the Treatment of Infections Caused by CRAB??"))

fim = time.time()
print(f"Tempo de execução: {fim - inicio:.4f} segundos")

A abordagem geral para o tratamento de infecções causadas por CRAB é administrar terapia combinada com pelo menos 2 agentes, até que seja observado uma resposta clínica adequada. É também sugerido que, no mínimo, um dos agentes na combinação seja baseado em sulbactam. A combinação mais preferida de sulbactam é a sulbactam-durlobactam em combinação com imipenem ou meropenem.

Além disso, é importante considerar os seguintes pontos:

* A falta de dados robustos que apoiem o tratamento de infecções por CRAB com qualquer agente individualmente demonstrado a ser eficaz in vitro;
* O uso de 2 agentes pode aumentar a probabilidade de administrar pelo menos um agente ativo;
* As altas cargas bacterianas esperadas nos casos de infecção por CRAB, devido à presença quase universal do patógeno.

É importante notar que existem muitos estudos observacionais que avaliam a eficácia da terapia combinada em comparação com a monoterapia para o tratamento de infecções por CRAB, com resultados variados.
Te

In [30]:
inicio = time.time()

print(generate_answer(llm, "What is the role of fluoroquinolones in the treatment of infections caused by S. maltophilia??"))

fim = time.time()
print(f"Tempo de execução: {fim - inicio:.4f} segundos")

Os fluoroquinolônios não parecem fornecer benefícios significativos como monoterapia, mas podem oferecer algum valor aditivo quando administrados em combinação com outros agentes.

É recomendado o uso de levofloxacina como componente de terapias combinadas, pois é uma opção para o tratamento de infecções causadas por S. maltophilia.
Tempo de execução: 44.3719 segundos


In [31]:
inicio = time.time()

print(generate_answer(llm, "What Is the Role of Cefiderocol for the Treatment of Infections Caused by S. maltophilia?"))

fim = time.time()
print(f"Tempo de execução: {fim - inicio:.4f} segundos")

O papel do cefiderocol no tratamento de infecções causadas por S. maltophilia é o uso como um componente da terapia combinada, pelo menos até que seja observado melhoria clínica. Ele é uma agente preferido para o tratamento de infecções por S. maltophilia, especialmente considerando a suscetibilidade de isolados de S. maltophilia ao cefiderocol, que se aproxima 100%.
Tempo de execução: 49.0402 segundos
